General concepts of working with Orbit, Ephem, and Phys objects
===============================================================

[DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) objects are used throughout sbpy to curate data. The sub-classes Orbit, Ephem, Obs and Phys are all derived from the same base class ([DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass)), meaning that they can all be used in the same way. 

The core of [DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) is an astropy [QTable](http://docs.astropy.org/en/stable/api/astropy.table.QTable.html#astropy.table.QTable) object, which is an astropy [Table](http://docs.astropy.org/en/stable/table/) that is aware of [units](http://docs.astropy.org/en/stable/units/). [DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) provides a number of convenience functions that make it easy to create these objects and populate them with data.

For some introductional examples on how to create, access, and modify [DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) objects, please have a look at the [documentation](https://sbpy.readthedocs.io/en/latest/sbpy/data/dataclass.html).

In the following sections, we provide some examples that are close to real use cases.

Adding a column and converting units
---------------------------

We obtain ephemerides for our target asteroid from JPL Horizons.

In [1]:
from astropy.time import Time
import astropy.units as u
from sbpy.data import Ephem

epoch1 = Time('2018-09-01 12:00', scale='utc')
epoch2 = Time('2018-09-30 12:00', scale='utc')
eph = Ephem.from_horizons('2018 RE3',
                          location='568',
                          epochs={'start': epoch1,
                                  'stop': epoch2,
                                  'step': 5*u.d},
                          skip_daylight=True)
eph.table

targetname,H,G,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA*cos(Dec)_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,epoch
,mag,,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,h,,mag,mag,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str10,float64,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object
(2018 RE3),27.3,0.15,,m,336.95173,0.97543,337.19193,1.07232,-62.6556,36.73963,238.9523,56.8561,535.08,-727.83,629238.4,-25629.9,20.303,0.341920379,1.193,0.13,22.37,99.355,0.0,612474.9,*,0.0,0.0,0.0,0.0,0.0,339.91,0.0,0.0,0.0,338.7724,0.6558,1.080246480628,-10.9442421,0.07201126546179,-10.3762343,0.598899,31.17143,10.42967,170.1319,/T,9.2133,71.4,66.8,0.6547,159.882,246.752,-9.20362,Aqr,69.182591,339.3053857,9.8643952,0.0,0.0,66.187038,-45.481247,1.6351407379,0.000355,1.147,1.194,1.556,0.567,-46.495,5.54,1.656,25272.6659,0.0248138,393.95,1431.79,0.168601,275.4816,1.862458291,9.2132,338.9039,5.2605,2458363.0
(2018 RE3),27.3,0.15,,,334.92564,2.96474,335.16448,3.06039,-144.843,97.00795,248.278,52.0563,438.58,-790.02,608405.1,-11847.2,46.843,0.6704726982,1.267,0.138,21.28,98.689,0.0,598721.5,*,0.0,0.0,0.0,0.0,0.0,313.45,0.0,0.0,0.0,343.3747,0.495,1.048792304321,-10.8271594,0.04189463679977,-10.0344392,0.348427,31.9528,10.14182,166.3115,/T,13.1465,140.2,14.1,0.5394,133.443,244.746,-11.75962,Peg,69.182518,338.1389549,12.4620776,0.0,0.0,66.363806,-42.645458,1.6625138305,0.000355,0.878,0.876,1.121,0.53,-44.881,3.73,1.24,14698.3858,0.024156,383.54,1393.94,0.098057,280.0861,2.326174278,13.1488,340.6555,6.4863,2458368.0
(2018 RE3),27.3,0.15,,,323.7034,13.42585,323.92995,13.51197,-1184.37,952.1053,270.9478,40.8584,242.67,-863.93,553839.9,32562.78,54.45,0.9990141954,1.526,0.166,19.21,92.936,0.0,535684.1,*,0.0,0.0,0.0,0.0,0.0,303.47,0.0,0.0,0.0,348.2595,0.3205,1.017784866055,-10.6370306,0.01285453701484,-9.1794149,0.106908,32.7566,9.98757,148.8011,/T,30.8237,129.6,4.4,0.3719,123.507,233.175,-25.29035,Peg,69.182458,331.1854466,26.2353386,0.0,0.0,66.958605,-27.406584,1.6912252381,0.000355,1.722,0.752,1.799,0.544,-17.609,6.15,1.879,4493.1707,0.0222909,354.37,1287.92,0.029975,284.987,3.403684467,30.8262,340.0626,13.4255,2458373.0
(2018 RE3),27.3,0.15,,,170.05247,11.50198,170.28638,11.40285,-429.792,-484.058,45.5732,-47.2685,654.97,605.32,-13722.5,31869.96,337.042,1.3275570429,999.0,0.0,36.0,0.73,0.0,34658.36,*,0.0,0.0,0.0,0.0,0.0,156.58,0.0,0.0,0.0,353.4521,0.1225,0.987477666912,-10.3069656,0.01819643837557,9.6748589,0.151335,33.59345,9.96172,9.6273,/L,170.1962,90.6,44.9,0.1807,336.562,107.76,-7.04649,Leo,69.182408,166.5888996,6.6392146,

Sky motion rates are provided for both RA and Dec in units of arcsec per hour:

In [2]:
print(eph['RA*cos(Dec)_rate', 'DEC_rate'])

<QTable length=6>
RA*cos(Dec)_rate  DEC_rate 
   arcsec / h    arcsec / h
    float64       float64  
---------------- ----------
        -62.6556   36.73963
        -144.843   97.00795
        -1184.37   952.1053
        -429.792   -484.058
         -39.095   -72.7217
        0.090585   -28.8419


Let's assume that we are interested in the absolute sky motion in units of arcsec per second. It would be handy to add a column to `eph` that contains this quantity. This can be easily done in the same way as for `astropy.table` [Table objects](https://docs.astropy.org/en/stable/table/). Finally, we want to print a table containing epoch, the distance to the observer, and the corresponding absolute sky motion:

In [3]:
import numpy as np
eph['abs_rate'] = np.sqrt(eph['RA*cos(Dec)_rate']**2 + eph['DEC_rate']**2).to('arcsec/second')
eph['epoch', 'delta', 'abs_rate']

<QTable length=6>
  epoch        delta             abs_rate      
                 AU             arcsec / s     
  object      float64            float64       
--------- ---------------- --------------------
2458363.0 0.07201126546179 0.020175779667146194
2458368.0 0.04189463679977  0.04842427349261143
2458373.0 0.01285453701484   0.4221159222171138
2458378.0 0.01819643837557   0.1798132842091663
2458383.0 0.04756413980326 0.022934529529610993
2458388.0 0.07789742321182 0.008011678403266262

Filtering table content
-----------------------

We have a list of 5 asteroids that we would like to observe on the night of 2018-09-12 at the Discovery Channel telescope. However, we don't want to observe these asteroids when they have an absolute sky motion rate faster than 0.1 arcsec/s and when the Moon is up.

In [4]:
import numpy as np
import astropy.units as u
from astropy.time import Time
from sbpy.data import Ephem

# target list
targets = ['2018 RR4', '2018 RE3', '2018 RC4', '2018 RQ2', '2018 RC1']

epoch1 = Time('2018-09-13 00:00', scale='utc')
epoch2 = Time('2018-09-14 00:00', scale='utc')
eph = Ephem.from_horizons(targets,
                          location='G37',
                          epochs={'start': epoch1,
                                  'stop': epoch2,
                                  'step': 10*u.minute},
                          skip_daylight=True)
print(len(eph.table))

345


A total of 345 ephemerides have been queried for the 5 asteroids. Now we apply the absolute sky motion rate (see example above) and moon filters; if the moon is up, `eph['flags']` will be set to `'m'` (see [this document](https://ssd.jpl.nasa.gov/?horizons_doc&table_quantities#table_quantities)):

In [5]:
eph = eph[np.sqrt(eph['RA*cos(Dec)_rate']**2 + eph['DEC_rate']**2).to('arcsec/second') < 0.1*u.arcsec/u.second]
eph = eph[eph['flags'] != 'm'] 

eph

targetname,H,G,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA*cos(Dec)_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,epoch
,mag,,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,h,,mag,mag,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str10,float64,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object
(2018 RC1),26.2,0.15,,,348.86634,4.40355,349.10606,4.50662,215.1766,-34.9955,112.7297,36.0393,582.58,680.86,639353.4,2434.567,13.524,20.045171878,1.696,0.259,20.04,99.486,0.0,617133.3,*,0.0,0.0,0.0,0.0,0.0,346.47,0.0,0.0,0.0,350.1328,0.3468,1.048789938227,-0.5637912,0.04294053138318,-0.5316125,0.357126,31.44401,1.9594,171.4259,/T,8.2243,135.1,14.6,0.3506,166.407,248.903,-8.14265,Psc,69.182438,351.7686515,8.4545903,0.0,0.0,82.97106,-50.867624,20.6378180929,0.000354,0.361,0.308,0.362,0.307,-8.49,0.7,0.475,14942.9668,0.0005703,9.06,32.93,0.099689,352.9121,-3.228565724,8.2233,350.8126,4.4012,2458374.666666667
(2018 RC1),26.2,0.15,,,348.87629,4.40193,349.11603,4.505,213.7809,-35.1119,114.7767,37.9161,594.81,670.24,639368.5,2438.305,13.507,20.2122948502,1.624,0.248,20.04,99.487,0.0,617151.4,*,0.0,0.0,0.0,0.0,0.0,346.49,0.0,0.0,0.0,350.1397,0.3466,1.048787678078,-0.5632519,0.04293842302127,-0.5196542,0.357108,31.44407,1.94477,171.4309,/T,8.2195,135.0,14.6,0.3504,166.424,248.903,-8.13755,Psc,69.182438,351.7772287,8.449178,0.0,0.0,82.982478,-50.874649,20.804522035,0.000354,0.36,0.308,0.362,0.307,-8.835,0.7,0.474,14942.6257,0.0005688,9.03,32.84,0.099686,352.9189,-3.062106922,8.2185,350.8204,4.3984,2458374.673611111
(2018 RC1),26.2,0.15,,,348.88619,4.4003,349.12593,4.50337,212.4431,-35.2328,116.9217,39.7615,608.15,658.19,639383.5,2442.024,13.49,20.3794178224,1.56,0.238,20.04,99.487,0.0,617169.4,*,0.0,0.0,0.0,0.0,0.0,346.5,0.0,0.0,0.0,350.1465,0.3463,1.048785420092,-0.5627126,0.04293636366828,-0.5071773,0.357091,31.44413,1.93053,171.436,/T,8.2147,134.9,14.7,0.3502,166.441,248.903,-8.13248,Psc,69.182438,351.7857448,8.4437857,0.0,0.0,82.993812,-50.881641,20.9712257834,0.000354,0.36,0.308,0.362,0.307,-9.157,0.7,0.474,14942.2856,0.0005673,9.01,32.74,0.099684,352.9257,-2.895643879,8.2137,350.8281,4.3956,2458374.680555556
(2018 RC1),26.2,0.15,,,348.89602,4.39866,349.13576,4.50174,211.1658,-35.3583,119.1755,41.5713,622.64,644.54,639398.2,2445.721,13.474,20.5465407947,1.504,0.229,20.04,99.488,0.0,617187.4,*,0.0,0.0,0.0,0.0,0.0,346.52,0.0,0.0,0.0,350.1534,0.3461,1.048783164269,-0.5621733,0.04293435535956,-0.4942043,0.357074,31.44419,1.91672,171.441,/T,8.2099,134.8,14.7,0.35,166.457,248.903,-8.12742,Psc,69.182438,351.794202,8.4384121,0.0,0.

As it turns out, only asteroid 2018 RC1 is observable under these conditions.

A tiny detail: note how `eph` is a `QTable` object after the filtering and not a `Ephem` object. This is because the filtering is really applied to the underlying `QTable` object. However, you can easily turn it back into an `Ephem` object by using [DataClass.from_table](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass.from_table).

Field Name Translations and Conversions
=======================================

Parameter names are usually not unique. For instance, `diameter` may also be referred to as `d` or `diam`, etc. sbpy acknowledges this ambiuity and provides internal translation tables for parameter names. Consider the following example:

In [6]:
from sbpy.data import Phys
import astropy.units as u
data = Phys.from_dict({'d': 10*u.km})
print(data.table)

 d  
 km 
----
10.0


In this `Phys` object, the diameter is defined as `d` and can be obtained as such. The internal translations also allow for queries with similar parameter labels:

In [7]:
data['d']

<Quantity [10.] km>

In [8]:
data['diameter']

<Quantity [10.] km>

In [9]:
data['D']

<Quantity [10.] km>

The permitted labels are defined in the list `sbpy.data.core.fieldnames`. A human readable version of this list is available in [the field name document](https://sbpy.readthedocs.io/en/latest/sbpy/data/fieldnames.html).

In addition to translations, sbpy is also using able to handle obvious property conversions, e.g., between and radius. Consider the following example:

In [10]:
data['radius']

<Quantity [5.] km>

Note that the object `data` does not contain a column `radius` before this query. However, the column is added as a result of the query:

In [11]:
data.table

d,radius
km,km
float64,float64
10.0,5.0
